In [1]:
import os
import json

In [2]:
%pwd

'c:\\Users\\rosha\\Documents\\Natural Language Processing\\End-to-end-NER\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rosha\\Documents\\Natural Language Processing\\End-to-end-NER'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path

In [6]:
from src.NER.constants import *
from src.NER.utils.common import read_yaml, create_directories

In [7]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
from src.NER.logging import logger
from src.NER.utils.common import get_size

In [9]:
import datasets

c:\Users\rosha\anaconda32\envs\NERend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def load_file(self):
        logger.info(f"Loading dataset from huggingface datasets library")
        data = datasets.load_dataset("conll2003")
        data_train = data['train']
        data_test = data['test']
        data_validation = data['validation']
        logger.info(f"printing the first line of tokens")
        print(data['train'][0])
        
        # path of loaded data
        file_path = self.config.local_data_file
        logger.info(f"path of loaded data : {file_path}")

        data_train.to_csv(file_path,index=False,header=True)
        data_test.to_csv(file_path,index=False,header=True)
        data_validation.to_csv(file_path,index=False,header=True)

        """# Ensure the directory for the file exists
        directory = os.path.dirname(file_path)
        os.makedirs(directory,exist_ok=True)
        # Save each example individually (you may modify this part based on your requirements)
        with open(file_path, 'w') as file:
             json.dump(data_train, file)
       """
        logger.info(f"Data loaded")

    

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.load_file()
    
except Exception as e:
    raise e

[2024-01-09 17:22:09,363: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-09 17:22:09,369: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-09 17:22:09,369: INFO: common: created directory at: artifacts]
[2024-01-09 17:22:09,371: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-09 17:22:09,373: INFO: 3943395081: Loading dataset from huggingface datasets library]


[2024-01-09 17:22:17,821: INFO: 3943395081: printing the first line of tokens]
{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}
[2024-01-09 17:22:17,823: INFO: 3943395081: path of loaded data : artifacts/data_ingestion/file]


Creating CSV from Arrow format: 100%|██████████| 4/4 [00:00<00:00,  5.46ba/s]

[2024-01-09 17:22:22,332: INFO: 3943395081: Data loaded]
